In [1]:
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor

In [2]:
data = pd.read_csv("../../../data/nasdaqlisted.txt", sep="|")

In [3]:
tickers = data[data["Market Category"] == "Q"]["Symbol"].tolist()

In [7]:
df = yf.download(tickers, interval="15m", start="2023-12-01", end="2024-01-28", threads=True)

[*********************100%%**********************]  1594 of 1594 completed

1588 Failed downloads:
['LYEL', 'PTLO', 'INSG', 'MRVI', 'CATY', 'CHTR', 'INNV', 'YMAB', 'EAR', 'TRIP', 'SWBI', 'GLPI', 'OLED', 'SND', 'AUROW', 'AAL', 'OKTA', 'MNMD', 'AGNCN', 'LUNG', 'MOLN', 'BPRN', 'MLYS', 'DLTR', 'XP', 'ROIC', 'MU', 'SWTX', 'LNW', 'GILT', 'LRFC', 'NTNX', 'COMM', 'VIRT', 'CIFR', 'ONBPO', 'FFIV', 'VYGR', 'BDTX', 'LASR', 'ATRA', 'TILE', 'ALVR', 'COCO', 'VLYPP', 'SIGI', 'NBTX', 'GEG', 'URBN', 'SNDX', 'JZ', 'PZZA', 'MGNI', 'INTR', 'YY', 'NRIM', 'RAIL', 'ACIW', 'CRVL', 'CSWC', 'FTAIM', 'APP', 'AOUT', 'SNPO', 'IBCP', 'PWP', 'MYNA', 'RRGB', 'FTRE', 'HOPE', 'HWCPZ', 'SRRK', 'MRSN', 'DRS', 'FANH', 'TRS', 'CZR', 'TLRY', 'AXNX', 'CMPR', 'NCMI', 'CRWD', 'SGHT', 'MATW', 'ENVX', 'TRMB', 'SIMO', 'MQ', 'PCTY', 'BRKL', 'EZPW', 'CPZ', 'FOXA', 'SATS', 'GT', 'GTX', 'FAST', 'VIR', 'HOFT', 'AY', 'IQ', 'UEIC', 'SRDX', 'VRTX', 'TIGR', 'PEGA', 'OB', 'PTEN', 'NVAX', 'BPYPN', 'KALU', 'YNDX', 'LYFT', 'BGFV', 'HYZNW', 'TE

In [22]:
yf.download("^GSPC", interval="15m", start="2023-12-01", end="2024-01-28")

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['^GSPC']: Exception('%ticker%: 15m data not available for startTime=1701406800 and endTime=1706418000. The requested range must be within the last 60 days.')


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [17]:
df[('Adj Close',   'AAPL')]

2023-12-19 09:30:00   NaN
2023-12-19 09:45:00   NaN
2023-12-19 10:00:00   NaN
2023-12-19 10:15:00   NaN
2023-12-19 10:30:00   NaN
                       ..
2024-01-26 14:45:00   NaN
2024-01-26 15:00:00   NaN
2024-01-26 15:15:00   NaN
2024-01-26 15:30:00   NaN
2024-01-26 15:45:00   NaN
Name: (Adj Close, AAPL), Length: 516, dtype: float64

In [ ]:
def getPR(symbol):
    sn = None
    pr = None
    try:
        stock = yf.Ticker(symbol)
        pr = stock.info['pegRatio']
        sn = stock.info['shortName']
    except Exception:
        pass
    return (sn, pr)


with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(getPR, sym): sym for sym in data['Symbol']}
    for future in concurrent.futures.as_completed(futures):
        sn, pr = future.result()
        if sn:
            print(f'{sn} : {pr}')